# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from category_encoders.cat_boost import CatBoostEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import make_scorer, mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import randint, uniform

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

# Lendo Bases

In [25]:
df_train = pd.read_csv('/home/jvcs/Área de Trabalho/regression_project/data/processed/train_dataset.csv')
df_test = pd.read_csv('/home/jvcs/Área de Trabalho/regression_project/data/processed/test_dataset.csv')

In [26]:
df_train

,date,bank,product,production,mean_bank,mean_product,mean_bp,std_bank,std_product,std_bp,lag_1,lag_7,lag_30,rolling_mean_7,rolling_mean_30,rolling_mean_60,rolling_std_7,rolling_std_30,rolling_std_60,day_of_week,month,is_month_end,fortnight,is_weekend,is_holiday,is_pre_holiday,is_post_holiday,diff_mean_lag1_bank,diff_mean_lag7_bank,diff_mean_lag30_bank,diff_mean_lag1_product,diff_mean_lag7_product,diff_mean_lag30_product,diff_mean_lag1_bank_product,diff_mean_lag7_bank_product,diff_mean_lag30_bank_product,ratio_mean_lag1_bank,ratio_mean_lag7_bank,ratio_mean_lag30_bank,ratio_mean_lag1_product,ratio_mean_lag7_product,ratio_mean_lag30_product,ratio_mean_lag1_bank_product,ratio_mean_lag7_bank_product,ratio_mean_lag30_bank_product,trend
0,2025-01-30,97,88011,42991.29,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,3,1,0,Second Fortnight,0,0,0,0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
1,2025-04-07,185,91108,1540.97,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0,4,0,First Fortnight,0,0,0,0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2,2025-05-08,46,31043,11820.76,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,3,5,0,First Fortnight,0,0,0,0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
3,2025-02-05,46,77090,1308.11,11820.760000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,2,2,0,First Fortnight,0,0,0,0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
4,2025-01-30,243,23237,4954.00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,3,1,0,Second Fortnight,0,0,0,0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60390,2025-05-27,62,14910,2355.01,24492.095613,6240.132979,6240.132979,5.294052e+04,1.059494e+04,1.059494e+04,5916.50,1843.45,2.258410e+03,5415.130000,7088.976333,6240.132979,2739.677505,1.291593e+04,1.059494e+04,1,5,0,Second Fortnight,0,0,0,0,-18575.595613,-22648.645613,-2.223369e+04,-323.632979,-4396.682979,-3.981723e+03,-323.632979,-4396.682979,-3.981723e+03,0.241568,0.075267,0.092210,0.948137,0.295418,0.361917,0.948137,0.295418,0.361917,47
60391,2025-04-15,208,28475,8499.74,50427.662775,21571.752766,21571.752766,2.465368e+05,7.505929e+04,7.505929e+04,45947.47,13609.05,1.053113e+04,17766.974286,15279.053333,42398.446000,14274.767681,1.009782e+04,1.305902e+05,1,4,0,First Fortnight,0,0,0,0,-4480.192775,-36818.612775,-3.989653e+04,24375.717234,-7962.702766,-1.104062e+04,24375.717234,-7962.702766,-1.104062e+04,0.911156,0.269873,0.208836,2.129983,0.630874,0.488191,2.129983,0.630874,0.488191,188
60392,2025-05-29,28,63829,59888.80,470052.573920,816885.581281,816885.581281,1.329092e+06,1.631493e+06,1.631493e+06,436294.04,100737.01,2.143934e+06,607158.571429,829757.301333,793715.381000,767500.549659,1.283643e+06,1.0

In [27]:
df_test

,date,bank,product,production,mean_bank,mean_product,mean_bp,std_bank,std_product,std_bp,lag_1,lag_7,lag_30,rolling_mean_7,rolling_mean_30,rolling_mean_60,rolling_std_7,rolling_std_30,rolling_std_60,day_of_week,month,is_month_end,fortnight,is_weekend,is_holiday,is_pre_holiday,is_post_holiday,diff_mean_lag1_bank,diff_mean_lag7_bank,diff_mean_lag30_bank,diff_mean_lag1_product,diff_mean_lag7_product,diff_mean_lag30_product,diff_mean_lag1_bank_product,diff_mean_lag7_bank_product,diff_mean_lag30_bank_product,ratio_mean_lag1_bank,ratio_mean_lag7_bank,ratio_mean_lag30_bank,ratio_mean_lag1_product,ratio_mean_lag7_product,ratio_mean_lag30_product,ratio_mean_lag1_bank_product,ratio_mean_lag7_bank_product,ratio_mean_lag30_bank_product,trend
0,2025-09-08,208,88025,16670.91,88346.827333,0.000000,0.000000,1.942443e+05,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,9,0,First Fortnight,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
1,2025-09-05,208,125683,14464.29,78316.618333,0.000000,0.000000,1.778162e+05,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4,9,0,First Fortnight,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2,2025-09-15,109,134554,3759.69,31989.358696,0.000000,0.000000,7.049434e+04,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,9,0,First Fortnight,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
3,2025-09-16,208,125688,28815.83,74955.969474,0.000000,0.000000,1.734260e+05,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,9,0,Second Fortnight,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
4,2025-09-16,260,1646,6150.09,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,9,0,Second Fortnight,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26968,2025-08-12,109,28908,4564.00,21671.735430,4139.769459,4139.769459,6.890827e+04,3500.097436,3500.097436,5850.00,500.00,866.00,5192.571429,4549.563667,4182.091500,4359.272564,3952.132246,3659.494808,1,8,0,First Fortnight,0,0,0,0,-15821.735430,-21171.735430,-20805.735430,1710.230541,-3639.769459,-3273.769459,1710.230541,-3639.769459,-3273.769459,0.269937,0.023072,0.039960,1.413122,0.120780,0.209190,1.413122,0.120780,0.209190,185
26969,2025-07-30,53,27355,21898.67,19602.739883,13230.017117,13230.017117,6.356490e+04,10993.278489,10993.278489,10120.95,12552.07,23612.95,14246.362857,14634.169000,13136.689833,5828.646754,13987.825213,11449.119879,2,7,0,Second Fortnight,0,0,0,0,-9481.789883,-7050.669883,4010.210117,-3109.067117,-677.947117,10382.932883,-3109.067117,-677.947117,10382.932883,0.516303,0.640322,1.204574,0.764999,0.948757,1.784801,0.764999,0.948757,1.784801,111
26970,2025-09-03,28,63828,4006.27,470324.162675,16963.010610,16963.010610,1.329432e+06,19756.602058,19756.602058,11261.04,133515.71,8334.84,32885.420000,20095.659333,20353.228833,46524.517440,26900.067943,24843.225873,2,9,0,First Fortnight,0,0,0,0,-459063.122675,-336808.452675,-461989.322675,-5701.970610,116552.699390,-8628.170610,-5701.970610,116552.699390,-8628.170610,0.023943,0.283880,0.017721,0.6638

# Pre-processamento

In [28]:
df_train.columns

Index(['date', 'bank', 'product', 'production', 'mean_bank', 'mean_product',
       'mean_bp', 'std_bank', 'std_product', 'std_bp', 'lag_1', 'lag_7',
       'lag_30', 'rolling_mean_7', 'rolling_mean_30', 'rolling_mean_60',
       'rolling_std_7', 'rolling_std_30', 'rolling_std_60', 'day_of_week',
       'month', 'is_month_end', 'fortnight', 'is_weekend', 'is_holiday',
       'is_pre_holiday', 'is_post_holiday', 'diff_mean_lag1_bank',
       'diff_mean_lag7_bank', 'diff_mean_lag30_bank', 'diff_mean_lag1_product',
       'diff_mean_lag7_product', 'diff_mean_lag30_product',
       'diff_mean_lag1_bank_product', 'diff_mean_lag7_bank_product',
       'diff_mean_lag30_bank_product', 'ratio_mean_lag1_bank',
       'ratio_mean_lag7_bank', 'ratio_mean_lag30_bank',
       'ratio_mean_lag1_product', 'ratio_mean_lag7_product',
       'ratio_mean_lag30_product', 'ratio_mean_lag1_bank_product',
       'ratio_mean_lag7_bank_product', 'ratio_mean_lag30_bank_product',
       'trend'],
      dtype='obje

In [29]:
df_train.sort_values(by=['date', 'bank', 'product'], inplace=True) 
df_test.sort_values(by=['date', 'bank', 'product'], inplace=True)

In [30]:
df_train['production'] = df_train['production'].clip(lower=0)
df_test['production'] = df_test['production'].clip(lower=0)
df_train['production'] = np.log1p(df_train['production']) # log(1+Y)
df_test['production'] = np.log1p(df_test['production']) # log(1+Y)

In [31]:
category_cols = ['bank', 'product', 'day_of_week', 'month', 'fortnight']
df_train[category_cols] = df_train[category_cols].astype('category')
df_test[category_cols] = df_test[category_cols].astype('category')

In [32]:
X_train = df_train.drop('production', axis=1)
y_train = df_train['production']
X_test = df_test.drop('production', axis=1)
y_test = df_test['production']

# Pipelines de Processamento

In [33]:
numerical_cols = X_train.drop(category_cols + ['date'], axis=1).columns.tolist()
categorical_cols = X_train[category_cols].columns.tolist()

# Pipeline para colunas numéricas
num_transf = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')), 
    ('scaler', MinMaxScaler())
])

# Pipeline para colunas categóricas (CRÍTICO: Target Encoding)
# Se você está usando CatBoostEncoder, ele deve ser treinado DENTRO do CV.
# O imputer é mantido, mas o scaler não é estritamente necessário para o encoder
cat_transf = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), 
    ('encoder', CatBoostEncoder(handle_missing='value', handle_unknown='value')),
    ('scaler', MinMaxScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transf, numerical_cols),
        ('cat', cat_transf, categorical_cols)
    ],
    remainder='drop' # Ignora colunas não especificadas (como 'date')
)


# Tuning de Hiperparâmetros

In [34]:
def safe_mape_loss(y_true, y_pred):
    """Calcula o MAPE, ignorando y_true = 0 na escala logarítmica."""
    # Como y_true agora é log(1+Y), y_true = 0 significa produção original = 0.
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    # Filtra os índices onde y_true não é zero (i.e., produção original > 0)
    non_zero_indices = y_true != 0 
    if np.sum(non_zero_indices) == 0:
        return 0.0
        
    y_true_filtered = y_true[non_zero_indices]
    y_pred_filtered = y_pred[non_zero_indices]
    
    # IMPORTANTE: Para calcular o MAPE corretamente, precisamos reverter para a escala ORIGINAL.
    # y_original = exp(y_log) - 1
    y_original_true = np.expm1(y_true_filtered)
    y_original_pred = np.expm1(y_pred_filtered)
    
    # Usamos np.abs(y_original_true) + 1e-6 no denominador para garantir a estabilidade
    return np.mean(np.abs(y_original_true - y_original_pred) / (np.abs(y_original_true) + 1e-6)) * 100

def rmse_score(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Scorer para uso no RandomizedSearchCV (quanto MENOR, MELHOR)
mape_scorer = make_scorer(safe_mape_loss, greater_is_better=False) 

# --- 4. TUNING COM PIPELINES CORRIGIDOS ---

# 4.1 Criação dos Pipelines Mestres
xgb_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(random_state=42)) 
])
rf_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Modelos (usando os pipelines)
models = {
    'xgb': xgb_pipe,
    'rf': rf_pipe
}

# 4.2 Hyperparameters (AJUSTADOS PARA O NOME DO PIPELINE)
params = {
    'xgb': {
        'regressor__n_estimators': randint(100, 500),
        'regressor__max_depth': randint(3, 20),
        'regressor__learning_rate': uniform(0.01, 0.29),
        'regressor__subsample': uniform(0.5, 1)
    },
    'rf': {
        'regressor__n_estimators': randint(100, 500),
        'regressor__max_depth': randint(3, 20),
        'regressor__min_samples_split': randint(2, 10),
        'regressor__min_samples_leaf': randint(1, 10)
    }
}

# 4.3 Scoring e CV
scoring = {
    'neg_mae': 'neg_mean_absolute_error',
    'neg_rmse': 'neg_mean_squared_error',
    'r2': 'r2',
    'neg_mape': mape_scorer 
}

cv = TimeSeriesSplit(n_splits=3)
grids = {}

# 4.4 Treinar modelos
for model_name, model in models.items():
    print(f"\n{'='*50}")
    print(f"TREINANDO {model_name.upper()}")
    print(f"{'='*50}")
    
    grids[model_name] = RandomizedSearchCV(
        estimator=model,
        param_distributions=params[model_name],
        cv=cv,
        scoring=scoring,
        refit='neg_mae', 
        n_jobs=-1,
        verbose=0,
        n_iter=10, 
        random_state=42
    )
    
    # O fit agora treina o pré-processador e o modelo a cada fold, sem leakage
    grids[model_name].fit(X_train, y_train) 
    
    # Resultados
    best_model = grids[model_name].best_estimator_
    best_index = grids[model_name].best_index_
    results = grids[model_name].cv_results_
    
    print(f"Melhores parâmetros: {grids[model_name].best_params_}")
    
    # Métricas no treino
    for score in scoring:
        # NOTE: MAPE precisa ser tratado manualmente para reverter o sinal e a escala
        if score == 'neg_mape':
            # MAPE: A métrica já reverteu a escala no scorer, mas o sinal precisa ser revertido
            mean_score = -results[f'mean_test_{score}'][best_index]
            std_score = results[f'std_test_{score}'][best_index]
            print(f"  MAPE (CV): {mean_score:.4f} (±{std_score:.4f})")
        else:
            mean_score = -results[f'mean_test_{score}'][best_index]
            std_score = results[f'std_test_{score}'][best_index]
            print(f"  {score.upper()}: {mean_score:.4f} (±{std_score:.4f})")

    # MAPE e RMSE manual (Para conferência, embora as métricas do CV sejam a fonte primária)
    train_pred = best_model.predict(X_train)
    print(f"  MAPE (Full Train): {safe_mape_loss(y_train, train_pred):.4f}")
    print(f"  RMSE (Log Scale): {rmse_score(y_train, train_pred):.4f}")

# Comparação final no teste
def compare_models(grids, X_test, y_test):
    print(f"\n{'='*50}")
    print("COMPARAÇÃO - CONJUNTO DE TESTE")
    print(f"{'='*50}")
    
    results = {}
    
    for model_name, grid in grids.items():
        best_model = grid.best_estimator_
        y_pred = best_model.predict(X_test)
        
        mae = mean_absolute_error(y_test, y_pred)
        rmse = rmse_score(y_test, y_pred)
        mape = mape_scorer(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results[model_name] = {
            'MAE': mae,
            'RMSE': rmse,
            'MAPE': mape,
            'R2': r2
        }
        
        print(f"\n🤖 {model_name.upper()}:")
        print(f"   MAE:   {mae:.4f}")
        print(f"   RMSE:  {rmse:.4f}")
        print(f"   MAPE:  {mape:.4f}")
        print(f"   R²:    {r2:.4f}")
    
    return results

# Uso:
# results = compare_models(grids, X_test, y_test)


TREINANDO XGB


Exception ignored in: Exception ignored in: <function _get_module_lock.<locals>.cb at 0x759978c69360>Could not import runpy module

Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 199, in cb
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 15, in <module>
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 187, in _run_module_as_main
<function _releaseLock at 0x70bb7e77f1c0>
Traceback (most recent call last):
  File "/usr/lib/python3.10/logging/__init__.py", line 228, in _releaseLock
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 187, in _run_module_as_main
  File "/usr/lib/python3.10/runpy.py", line 187, in _run_module_as_main
KeyboardInterrupt: 
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 187, in _run_module_as_main
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 18

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGINT(-2), SIGINT(-2), SIGINT(-2), SIGINT(-2), SIGINT(-2), SIGINT(-2), EXIT(1), SIGINT(-2)}